In [ ]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from glob import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
N_JOBS = 2 # Set number of cores you have available.

train_files = glob('../input/seti-breakthrough-listen/train/*/*.npy')
test_files = glob('../input/seti-breakthrough-listen/test/*/*.npy')

len(train_files), len(test_files)

In [ ]:
!mkdir train_norm
!mkdir test_norm

def proc_images(fn='', folder='train_norm'):
    img = np.load(fn).astype(np.float32)
    
    for i in range(6):
        img[i,:,:] = img[i,:,:] - np.mean(img[i,:,:])
        img[i,:,:] = img[i,:,:] / np.std(img[i,:,:])
        
    file = fn.split('/')[-1]
    np.save( folder + file, img.astype(np.float16) )


res = Parallel(n_jobs=N_JOBS,backend='multiprocessing')(
    delayed(proc_images)(fn=fn, folder='train_norm/')
        for fn in tqdm(train_files[:1000])
) # remove [:1000] to process all the local files.

res = Parallel(n_jobs=N_JOBS,backend='multiprocessing')(
    delayed(proc_images)(fn=fn, folder='test_norm/') 
        for fn in tqdm(test_files[:1000])
) # remove [:1000] to process all the local files.

In [ ]:
train_norm_files = glob('train_norm/*.npy')
test_norm_files = glob('test_norm/*.npy')
len(train_norm_files), train_norm_files[:10]

In [ ]:
# Check differences between original and normalized
image = np.load('../input/seti-breakthrough-listen/train/7/7072ba75fac0.npy').astype(np.float32)
image_norm = np.load('train_norm/7072ba75fac0.npy').astype(np.float32)

print(np.mean(image,2).mean(1) - np.mean(image_norm,2).mean(1) )
print( )
print( np.std(image,2).mean(1) - np.std(image_norm,2).mean(1) )

In [ ]:
plt.imshow(np.hstack(image[[0,2,4]]))

In [ ]:
plt.imshow(np.hstack(image_norm[[0,2,4]]))